In [3]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union

Given

$$\boldsymbol{r}(\boldsymbol{x})=\begin{bmatrix}x_1\\ \frac{10x_1}{x_1+0.1}+2x_2^2\end{bmatrix}$$

$$r_1 = x_1 = 0$$

If $x_1=0$ then $r_2=2x_2^2$, which can only be $0$ if $x_2=0$


Jacobian of $\boldsymbol{r}$

$$J_\boldsymbol{r}=\begin{bmatrix}1 & 0\\\frac{10}{x_1+0.1}-\frac{10x_1}{(x_1+0.1)^2}&4x_2\end{bmatrix}$$

Determinant is $0$ at $(0,0)$ as such singular

In [10]:
def linearLSQ(A, y):
    Q, R = np.linalg.qr(A, mode='reduced')
    x = np.linalg.solve(R, np.dot(Q.T, y))

    return x

In [11]:
def rJ(x: Union[list, np.ndarray]):
    r = np.array([x[0], (10*x[0]) / (x[0] + 0.1) + 2*x[1]**2])
    J = np.array([[1, 0],[10/(x[0] + 0.1) - (10*x[0]) / ((x[0]+0.1)**2), 4*x[1]]])
    
    return r, J

r, J = rJ(np.array([1,1]))
r, J

(array([ 1.        , 11.09090909]),
 array([[1.        , 0.        ],
        [0.82644628, 4.        ]]))

In [13]:
def LM(func, x0, *args):
    maxit = 100 * len(x0)
    tol = 1.0e-10

    # Initial iteration
    x = np.array(x0)
    it = 0
    rx, Jx = func(x, *args)
    f = 0.5 * np.linalg.norm(rx)**2
    df = np.dot(Jx.T, rx)
    converged = (np.linalg.norm(df, np.inf) <= tol)
    nfun = 1

    # Initial lambda
    lambda_val = np.linalg.norm(np.dot(Jx.T, Jx))

    # Store data for plotting
    X = [x]
    F = [f]
    dF = [df]

    while not converged and it < maxit:
        it += 1
        
        # Calculate the search direction by solving a linear LSQ problem
        A = np.vstack([Jx, np.sqrt(lambda_val)*np.eye(len(x))])
        b = np.hstack([-rx, np.zeros_like(x)])
        #p = np.linalg.lstsq(A, b, rcond=None)[0]
        p = linearLSQ(A, b)
        
        # Update the iterate, Jacobian, residual, f, df
        x_new = x + p
        rx_new, Jx_new = func(x_new, *args)
        f_new = 0.5 * np.linalg.norm(rx_new)**2

        rho = (f - f_new) / (0.5 * np.dot(p, (lambda_val * p - np.dot(Jx.T, rx))))
        if rho > 0.75:
            lambda_val /= 3
        elif rho < 0.25:
            lambda_val *= 2
        
        # Accept or reject x_new
        if rho > 0:
            x = x_new
            rx = rx_new
            f = f_new
            Jx = Jx_new
            df = np.dot(Jx.T, rx)

        converged = (np.linalg.norm(df, np.inf) <= tol)
        nfun += 1

        # Store data for plotting
        X.append(x)
        F.append(f)
        dF.append(df)

    # Prepare return data
    if not converged:
        x = None  # Use None instead of empty for Python

    stat = {
        "converged": converged,
        "iter": it,
        "X": np.array(X),
        "F": np.array(F),
        "dF": np.array(dF),
        "nfun": nfun
    }
    
    return x, stat

LM(rJ, np.array([3,1]))

(None,
 {'converged': False,
  'iter': 200,
  'X': array([[ 3.00000000e+00,  1.00000000e+00],
         [ 2.78834176e+00, -4.54982757e-01],
         [ 2.56619481e+00,  4.89185657e-01],
         [ 2.45031495e+00, -6.04048370e-02],
         [ 2.33173824e+00,  1.17338634e-02],
         [ 1.99455901e+00, -3.00886943e-02],
         [ 1.09420765e+00,  2.83250036e-01],
         [ 1.09420765e+00,  2.83250036e-01],
         [ 1.09420765e+00,  2.83250036e-01],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [ 7.77676447e-02, -1.33268617e+00],
         [-3.38666266e-02, -1.31387572e+00],
         [-2.66290237e-02, -1.31404057e+00],
         [-2.56853228e-02, -1.31406190e+00],
      

In [1]:
def Levenberg_Marquardt_yq(func, x0, tau, *args):
    # Solver settings and info
    maxit = 100 * len(x0)
    tol = 1.0e-10

    # Initial iteration
    x = np.array(x0)
    it = 0
    rx, Jx = func(x, *args)
    f = 0.5 * np.linalg.norm(rx)**2
    df = np.dot(Jx.T, rx)
    converged = (np.linalg.norm(df, np.inf) <= tol)
    nfun = 1

    # Initial lambda
    lambda_val = tau * np.linalg.norm(np.dot(Jx.T, Jx))
    nu = 2

    # Store data for plotting
    X = [x]
    F = [f]
    dF = [df]

    # Main loop of L-M method
    while not converged and it < maxit:
        it += 1
        
        # Calculate the search direction by solving a linear LSQ problem
        A = np.vstack([Jx, np.sqrt(lambda_val)*np.eye(len(x))])
        b = np.hstack([-rx, np.zeros_like(x)])
        #p = np.linalg.lstsq(A, b, rcond=None)[0]
        p = linearLSQ(A, b)
        
        # Update the iterate, Jacobian, residual, f, df
        x_new = x + p
        rx_new, Jx_new = func(x_new, *args)
        f_new = 0.5 * np.linalg.norm(rx_new)**2

        rho = (f - f_new) / (0.5 * np.dot(p, (lambda_val * p - np.dot(Jx.T, rx))))
        if rho > 0.75:
            lambda_val /= 3
        elif rho < 0.25:
            lambda_val *= 2
        
        # Accept or reject x_new
        if rho > 0:
            x = x_new
            rx = rx_new
            f = f_new
            Jx = Jx_new
            df = np.dot(Jx.T, rx)

            lambda_val = lambda_val * max(1/3, 1 - (2 * rho - 1)**3)
        else:
            lambda_val = lambda_val * nu
            nu = 2 * nu


        converged = (np.linalg.norm(df, np.inf) <= tol)
        nfun += 1

        # Store data for plotting
        X.append(x)
        F.append(f)
        dF.append(df)

    # Prepare return data
    if not converged:
        x = None  # Use None instead of empty for Python

    stat = {
        "converged": converged,
        "iter": it,
        "X": np.array(X),
        "F": np.array(F),
        "dF": np.array(dF),
        "nfun": nfun
    }
    
    return x, stat

In [8]:
x, stats = Levenberg_Marquardt_yq(rJ, np.array([3, 1]), tau=1)
stats

{'converged': False,
 'iter': 200,
 'X': array([[ 3.00000000e+00,  1.00000000e+00],
        [ 2.78834176e+00, -4.54982757e-01],
        [ 2.57144964e+00,  4.68535288e-01],
        [ 2.50645733e+00,  1.67197548e-01],
        [ 2.43687843e+00,  5.12833438e-02],
        [ 2.35748902e+00,  1.06915037e-02],
        [ 1.79492024e+00, -5.71606859e-02],
        [ 1.79492024e+00, -5.71606859e-02],
        [ 1.79492024e+00, -5.71606859e-02],
        [ 1.59688314e+00,  4.38275750e-02],
        [ 6.57558508e-01, -3.48068451e-01],
        [ 6.57558508e-01, -3.48068451e-01],
        [ 6.57558508e-01, -3.48068451e-01],
        [ 5.88731957e-01, -2.95124125e-01],
        [ 1.98236841e-03,  3.33948289e-02],
        [-6.18579224e-05,  3.33919586e-02],
        [-2.23101023e-05,  3.33919168e-02],
        [-2.22920673e-05,  3.33910661e-02],
        [-2.22818517e-05,  3.33834136e-02],
        [-2.21903326e-05,  3.33148389e-02],
        [-2.13993605e-05,  3.27208871e-02],
        [-2.13993605e-05,  3.2720887